# HandsOn Week 13
Welcome to HandsOn Week 13. In this HandsOn, you will try to play with spark streaming where the data is from a Kafka producer.

## Milestone 1
In this milestone, you need to setup Zookeeper and Kafka in your computer/VM you use to run Spark. See [here](https://towardsdatascience.com/kafka-python-explained-in-10-lines-of-code-800e3e07dad1) for one of the tutorials and [here](https://tecadmin.net/install-apache-kafka-ubuntu/) for additional one to ubuntu users. Then, you also need to install kafka-python by ```pip install kafka-python```.<br><br>
In this milestone, you only need to run ```producer_variance.py``` and ```consumer_variance.py``` (these two code files are already provided inside the zip file).

Screenshot your ```consumer_variance.py``` output, and put in this cell below. 

<img title="Output Consumer" align="left" src="Week13_M1.png" alt="Drawing" style="width: 850px"/>

## Milestone 2
After making sure that the message is published by ```producer_variance.py``` and successfully consumed by ```consumer_variance.py``` in the topic of ```variance``` in the Milestone 1 above, then, you are ready for Milestone 2.<br>

In Milestone 2, you need to implement ```calculate_variance``` function with the formula --> $variance = \frac{\sum_{i=1}^{N}x_i^2}{N}-(\frac{\sum_{i=1}^{N}x_i}{N})^2$. This function will be used to calculate variance for each window operation to the streaming data, and the variance is "**accumulative/global**" value up to current stream data. For example, in the first window, we have data ```1,2,3```, then the variance is the variance of ```1,2,3```. Let's say we have streaming data of ```4,5,6``` in the second window, thus the variance in this second window is the variance of ```1,2,3,4,5,6```, and so on for the following windows.<br>

The ```calculate_variance``` function will return a DStream (RDD) with a format of ```('sum_x_square:', sum_x_square_value, 'sum_x:', sum_x_value, 'n:', n_value, 'var:', variance_value)``` where ```sum_x_square_value```$=\sum_{i=1}^{N}x_i^2$, ```sum_x_value```$=\sum_{i=1}^{N}x_i$ and ```n```$=N$. Note that $x_i=$ i-th of individual stream data, and $N=$ the number of individual stream data -count- up to i-th data.

**Important:** In order to stream from Kafka producer to Spark Streaming, you need to download [spark-streaming-kafka-0-8-assembly_2.11-2.4.5.jar](https://mvnrepository.com/artifact/org.apache.spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.5) from maven repository (adjust with your Spark version), and put this jar file to ```your_spark_folder/jars```. For VM provided by the class, ```spark_folder``` is in ```/home/bigdata/spark-2.4.5-bin-hadoop2.7```.

start-dfs.sh
start-yarn.sh
sudo systemctl start zookeeper
sudo systemctl start kafka
pyspark

In [1]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

KAFKA_TOPIC = "variance"
BOOTSTRAP_SERVER = "localhost:9092"

ssc = StreamingContext(sc,1) #stream each one second
ssc.checkpoint("./checkpoint")
lines = KafkaUtils.createDirectStream(ssc, [KAFKA_TOPIC],
                                      {"metadata.broker.list": BOOTSTRAP_SERVER})

def calculate_variance(lines, window_length = 2, sliding_interval = 2):
    """
    Function to calculate "accumulated/global variance" in each window operation
    Params:
        lines: Spark DStream defined above (in this jupyter cell)
        window_length: length of window in windowing operation
        sliding_interval: sliding interval for the window operation
    Return:
        result: DStream (RDD) of variance result with 
                format --> ('sum_x_square:', sum_x_square_value, 'sum_x:', sum_x_value, 'n:', n_value, 'var:', variance_value)
                Example:   ('sum_x_square:', 182.0, 'sum_x:', 42.0, 'n:', 12.0, 'var:', 2.916666666666666)
    """
    
    # Realize this function here. Note that you are not allowed to modify any code other than this function.    
    temp = lines.window(window_length, sliding_interval).map(lambda x: (x[1].split(' ')))
    
    sum_x_square_value = temp.flatMap(lambda x: (int(i) ** 2 for i in x)).reduce(lambda x, y: x + y)
    sum_x_value = temp.flatMap(lambda x: (int(i) for i in x)).reduce(lambda x, y: x + y)
    n_value = temp.flatMap(lambda x: (1 for i in x)).reduce(lambda x, y: x + y)
    
    temp1 = sum_x_square_value.map(lambda x: (1, x))
    temp2 = sum_x_value.map(lambda x: (1, x))
    temp3 = n_value.map(lambda x: (1, x))    
    temp4 = temp1.join(temp2).join(temp3)
    variance_value = temp4.map(lambda x: (x[1][0][0] / x[1][1] - (x[1][0][1] / x[1][1]) ** 2))
    
    result1 = sum_x_square_value.map(lambda x: (1, ('sum_x_square:', x)))
    result2 = sum_x_value.map(lambda x: (1, ('sum_x:', x)))
    result3 = n_value.map(lambda x: (1, ('n:', x)))
    result4 = variance_value.map(lambda x: (1, ('var:', x)))
    result5 = result1.join(result2).join(result3).join(result4)
    result = result5.map(lambda x: ((x[1][0][0][0][0], x[1][0][0][0][1], x[1][0][0][1][0], x[1][0][0][1][1], x[1][0][1][0], x[1][0][1][1], x[1][1][0], x[1][1][1])))
    
    return result

# run the function
result = calculate_variance(lines, window_length=2, sliding_interval=2)
# Print
result.pprint()
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2020-04-14 03:58:19
-------------------------------------------
('sum_x_square:', 273, 'sum_x:', 63, 'n:', 18, 'var:', 2.916666666666666)

-------------------------------------------
Time: 2020-04-14 03:58:21
-------------------------------------------
('sum_x_square:', 91, 'sum_x:', 21, 'n:', 6, 'var:', 2.916666666666666)

-------------------------------------------
Time: 2020-04-14 03:58:23
-------------------------------------------
('sum_x_square:', 182, 'sum_x:', 42, 'n:', 12, 'var:', 2.916666666666666)

-------------------------------------------
Time: 2020-04-14 03:58:25
-------------------------------------------
('sum_x_square:', 182, 'sum_x:', 42, 'n:', 12, 'var:', 2.916666666666666)

-------------------------------------------
Time: 2020-04-14 03:58:27
-------------------------------------------
('sum_x_square:', 182, 'sum_x:', 42, 'n:', 12, 'var:', 2.916666666666666)

-------------------------------------------
Time: 2020-04

KeyboardInterrupt: 

## Submission
Archive this ipynb file and the screenshot image needed in the Milestone 1 into zip file with a format of: ```HandsOnWeek13_NIM_FullName.zip```, and submit to the course portal.

**Note**: make sure in the Milestone 2, the cell has its output, but not too many streams (you can save this ipynb file approximatelly in the range of 4-20 window operations)

Enjoy...